# Análisis de Investigación Macrozona Sur
## Datos desde CNED Chile
* Ejecutado en mi entorno conda Stats
* Autor: Alexis Salas Burgos 
* Fecha: Septiembre 2020

In [ ]:
# importando bibliotecas requerido en python
import pandas as pd
import altair as alt
import seaborn as sns
import json

In [ ]:
df = pd.read_csv("../data/DataCiencia_disciplinas_data.csv", sep=',', skiprows=0)
df.rename(columns={"CATEGORÍA":"Disciplina WoS"})
# process json-like elemnts and NaN values for each column
for n in df.columns[1:]:
    col_isna = df[n].isna()
    # convert json-like string elements to json object
    col_tojson = df.loc[-col_isna,n].str.replace("\'","\"").apply(json.loads)
    # extract value from json object
    df.loc[-col_isna,n] = col_tojson.apply(lambda item: item["times"])
    # nan elements to default zero
    df.loc[col_isna,n] = 0


In [ ]:
pd_long = df.melt(id_vars=["CATEGORÍA"],var_name="Disciplina OECD",value_name="Cantidad")
pd_long

In [ ]:
# import altair as alt
# alt.data_transformers.disable_max_rows()

# alt.Chart(df_log).mark_bar(
#     cornerRadiusTopLeft=3,
#     cornerRadiusTopRight=3
# ).encode(
#     y=alt.Y('sum(OECD_AREA):Q', stack=True),
#     x='OECD_AREA:O',
#     color='GEN_ALU:N',
#     column='REGION_SEDE:O'
    
# )

# Plots Data Reportes

### Datos concatenads al scrapear individualmente por año.

In [ ]:
datos_region = pd.read_csv("../data/DataCiencia_reporteRegion_sur.v2.csv",index_col=0)
datos_oecd = pd.read_csv("../data/DataCiencia_reporteOECD_sur.v2.csv",index_col=0)
datos_wos = pd.read_csv("../data/DataCiencia_reporteWoS_sur.v2.csv",index_col=0)
datos_inst = pd.read_csv("../data/DataCiencia_reporteInst_sur.v2.csv",index_col=0)

In [ ]:


# datos_region["max_sexo"] = 100

# sexo1 = alt.Chart(datos_region).mark_bar(
# ).encode(
#     y2 = alt.Y2('max_sexo:Q',title="Porcentaje"),
#     y = alt.Y('% Mujer:Q',title=""),
#     x='año:O',
#     color=alt.value('blue'),
#     #column='Nombre Región:N'
    
# )

# sexo2 = alt.Chart(datos_region).mark_bar(
# ).encode(
#     y = alt.Y('% Mujer:Q',title=""),
#     x='año:O',
#     color=alt.value('red'),
#     #column='Nombre Región:N'
    
# )
# final1= (sexo1+sexo2).properties( width=200,
#     height=200).facet(alt.Facet('region:N', title="Publicaciones en Regiónes por Sexo")).resolve_scale(
#   x='independent'
# )
# final1
# final1.save("Pub_sexo_regiones.svg",scale_factor=1.0)

In [ ]:
# produccion per capita
datos_region.loc[datos_region['año']!=2020].groupby(['Región'])['Producción Per Cápita'].describe()

In [ ]:
#alt.data_transformers.disable_max_rows()

alt.Chart(datos_region.loc[datos_region['año']!=2020]).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    y = alt.Y('# Publicaciones:Q'),
    x='año:O',
    color='region:N',
    #column='Nombre Región:N'
    
).properties( width=200,
    height=200).facet(facet='region:O', columns=2,center=True).resolve_scale(
  x='independent'
)

In [ ]:
alt.Chart(datos_region.loc[datos_region['año']!=2020]).mark_line(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    y = alt.Y('Producción Per Cápita:Q'),
    x='año:O',
    color='region:N',
    #column='Nombre Región:N'
    
).properties( width=700,
    height=300)


### OECD pulicaciones

In [ ]:
datos_oecd["% Mujer"] = datos_oecd["% Mujer"].str.strip("%").copy().astype(int)
datos_oecd["% Hombre"] = datos_oecd["% Hombre"].str.strip("%").copy().astype(int)

datos_oecd["max_sexo"] = 100


In [ ]:
datos_oecd.loc[datos_oecd['año']!=2020].groupby(["Disciplina","region",'año'])['# Publicaciones'].describe()

In [ ]:
datos_oecd_aggmean = datos_oecd.loc[datos_oecd['año']!=2020].groupby(["Disciplina","region"]).mean().reset_index()
datos_oecd_aggmean.head()

### Descripcion por sexo

In [ ]:
datos_oecd_aggmean["Pub. Mujer"] = round(((datos_oecd_aggmean["% Mujer"]/100)*datos_oecd_aggmean["# Publicaciones"]))
datos_oecd_aggmean["Pub. Hombre"] = round(((datos_oecd_aggmean["% Hombre"]/100)*datos_oecd_aggmean["# Publicaciones"]))

In [ ]:
datos_oecd_aggmean.head()

In [ ]:
datos_oecd_pubsexo = pd.melt(datos_oecd_aggmean,id_vars=['Disciplina','region'], value_vars = ['Pub. Mujer','Pub. Hombre'],
                             value_name="Publicaciones",var_name="Sexo")
datos_oecd_pubsexo.head()

In [ ]:

pubsexo = alt.Chart(datos_oecd_pubsexo).mark_bar(
).encode(
    y = alt.Y('Publicaciones:Q',title="Publicaciones"),
    x=alt.X('region:O',title="Región"),
    color=alt.Color('Sexo',title=""),
    #column=alt.Column('Disciplina:O'),
    ).properties( width=150,
    height=150).facet(alt.Facet('Disciplina:O',title=""), columns=3,center=True).resolve_scale(
  y='independent'
)
pubsexo
#pubsexo.save("pubsexo_oecd.png",scale_factor=2.0)

### produccion per capita

In [ ]:
alt.Chart(datos_oecd_aggmean).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    y = alt.Y('Producción Per Cápita:Q'),
    x=alt.X('region:O',title="Región",),
    color='Disciplina:N',
    #column='Disciplina:N'
    
).properties( width=100,
    height=200).facet(alt.Facet('Disciplina:O',title=""), columns=3,center=True)

In [ ]:


# sexo1 = alt.Chart(datos_oecd).mark_bar(
# ).encode(
#     y2 = alt.Y2('max_sexo:Q',title="Porcentaje"),
#     y = alt.Y('% Mujer:Q',title=""),
#     x='año:O',
#     color=alt.value('blue'),
#     #column='Nombre Región:N'
    
# )

# sexo2 = alt.Chart(datos_oecd).mark_bar(
# ).encode(
#     y = alt.Y('% Mujer:Q',title=""),
#     x='año:O',
#     color=alt.value('red'),
#     #column='Nombre Región:N'
    
# )
# final1= (sexo1+sexo2).properties( width=200,
#     height=200).facet(alt.Facet('Disciplina:N', title="Publicaciones Disciplinas OECD por Sexo"),columns=3).resolve_scale(
#   x='independent'
# )
# final1

In [ ]:
alt.Chart(datos_oecd).mark_line(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    y = alt.Y('# Publicaciones:Q'),
    x='año:O',
    color='Disciplina:N',
    #column='Nombre Región:N'
    
).properties( width=700,
    height=300)

## Publicaciones WoS

In [ ]:
# mapping OECD WoS
oecd_wos_mapping = pd.read_table("../data/oecd_wos_mapping.txt",names=["OECD","CODE","WoS"])
oecd_wos_mapping = oecd_wos_mapping[["OECD","WoS"]]

oecd_wos_dict = {n:g["OECD"].tolist()[0] for n,g in oecd_wos_mapping.groupby("WoS")}
datos_wos["OECD"] = datos_wos["Disciplina"].map(oecd_wos_dict)

In [ ]:
habitantes = {"Biobío":1556805,"Ñuble":480609,"Maule":1044950,"O'Higgins":914555}

In [ ]:
datos_wos_aggmean = datos_wos.loc[datos_wos['año']!=2020].groupby(["Disciplina","region","OECD"]).mean().reset_index()


In [ ]:
datos_wos_aggmean["Pub/100mil"] = datos_wos_aggmean["# Publicaciones"]/datos_wos_aggmean["region"].map(habitantes)*100000

### Descripcion por region

In [ ]:
biobio_disc = set(datos_wos_aggmean.loc[datos_wos_aggmean['region']=='Biobío']['Disciplina'])
wos_disc = set(oecd_wos_mapping["WoS"].tolist())
len(wos_disc)

In [ ]:
# Bio Bio
datos_wos_aggmean.loc[datos_wos_aggmean['region']=='Biobío'].groupby(['Disciplina','region','OECD'])['Pub/100mil'].describe().sort_values(by='mean',ascending=False)

In [ ]:
# maule
datos_wos_aggmean.loc[datos_wos_aggmean['region']=='Maule'].groupby(['Disciplina','region','OECD'])['Pub/100mil'].describe().sort_values(by='mean',ascending=False)

In [ ]:
# ohiggins
datos_wos_aggmean.loc[datos_wos_aggmean['region']=="O'Higgins"].groupby(['Disciplina','region','OECD'])['Pub/100mil'].describe().sort_values(by='mean',ascending=False)

In [ ]:
# ñuble
datos_wos_aggmean.loc[datos_wos_aggmean['region']=="Ñuble"].groupby(['Disciplina','region','OECD'])['Pub/100mil'].describe().sort_values(by='mean',ascending=False).head(10)

In [ ]:
sort_list = oecd_wos_mapping["WoS"].tolist()

#alt.data_transformers.disable_max_rows()

wos_oecd_pub100 = alt.Chart(datos_wos_aggmean).mark_bar(
).encode(
    y=alt.Y('Pub/100mil:Q',title="Pub/100mil"),
    x=alt.X('Disciplina:O',title="Disciplina",sort=sort_list),
    color=alt.Color('OECD',title=""),
    #column=alt.Column('region:O'),
    ).properties( width=2000,
    height=300).facet(alt.Facet('region:O',title=""), columns=1,center=True).interactive()

#wos_oecd_pub100.save("wos_oecd_pub100.png",scale_factor=2.0)
wos_oecd_pub100

### instituciones

In [ ]:
datos_inst_2008_ = datos_inst.loc[datos_inst["año"]==2020]

In [ ]:
order_inst = datos_inst_2008_["Institución"].sort_values().unique()

In [ ]:
tupl_inst = list()
for i,ins in enumerate(order_inst):
    if i <= len(order_inst)/2:
        tupl_inst.append((ins,1))
    else:
        tupl_inst.append((ins,2))

In [ ]:
map_split_inst = dict(tupl_inst)

In [ ]:
datos_inst_2008_.loc[:,"group"] = datos_inst_2008_["Institución"].map(map_split_inst)

In [ ]:
datos_inst_2008_.columns

In [ ]:
bar1 = alt.Chart(datos_inst_2008_).mark_bar(
        #cornerRadiusTopLeft=3,
        #cornerRadiusTopRight=3
).encode(
    y = alt.Y('Producción Per Cápita:Q',title="Producción per cápita",
              axis={"titleFontSize":16,"labelFontSize":16}),
    x=alt.X('Institución:O',title="Instituciones",axis={"labelAngle":-90, "titleFontSize":16,"labelFontSize":16}),
    color='region:N',
    row=alt.Row('group:N',title=None,header=alt.Header(labels=False))
    
).properties(
    title="Producción per cápita de Instituciones Chilenas Macrozona Centro-Sur desde 2008"
    #width=200,
    #height=200
).resolve_scale(
    x='independent',
    #y='independent'
).configure_title(
    fontSize=16,
    #font='Courier',
    anchor='middle',
    color='black'
)
bar1

## Proyectos, universidades

In [ ]:
hojas = ["F. REGULAR", "F. Iniciación","F.Postdoctorado","FONDAP","FONDEF","FONDEQUIP","FONIS","PIA"]

In [ ]:
# 2014-2019
fregular = pd.read_excel("../data/Estudio universidades 2014-2019.xlsx",sheet_name=hojas[0],skiprows=3,skipfooter=2).dropna(axis=1)
fregular.head()

In [ ]:
fregularm = pd.melt(fregular,id_vars=['Institución'], value_vars = ['Postulados','Adjudicados'])


bar1 = alt.Chart(fregularm).mark_bar(
).encode(
    y = alt.Y('value:Q',title=""),
    x='Institución:O',
    color=alt.Color('variable',title=""),
    #column=alt.Column('variable'),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'variable',
      sort='ascending'
    )
)


line = alt.Chart(fregular).mark_line(stroke="red"
).encode(
    y = alt.Y('Tasa de Aprobación:Q',axis=alt.Axis(title='Tasa de Aprobación', titleColor='red',grid=True)),
    x='Institución:O',    
)
final = alt.layer(bar1, line).resolve_scale(
    y = 'independent',
    #color="independent"
)
final
#final.save("fregular.svg",scale_factor=1.0)



In [ ]:
finiciacion = pd.read_excel("../../data/Estudio universidades 2014-2019.xlsx",sheet_name=hojas[1],skiprows=4,skipfooter=2).dropna(axis=1)
finiciacion.head()

In [ ]:
finiciacionm = pd.melt(finiciacion,id_vars=['Institución'], value_vars = ['Postulados','Adjudicados'])


bar1 = alt.Chart(finiciacionm).mark_bar(
).encode(
    y = alt.Y('value:Q',title=""),
    x='Institución:O',
    color=alt.Color('variable',title=""),
    #column=alt.Column('variable'),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'variable',
      sort='ascending'
    )
)


line = alt.Chart(finiciacion).mark_line(stroke="red"
).encode(
    y = alt.Y('Tasa de Aprobación:Q',axis=alt.Axis(title='Tasa de Aprobación', titleColor='red',grid=True)),
    x='Institución:O',    
)
final = alt.layer(bar1, line).resolve_scale(
    y = 'independent',
    #color="independent"
)
final